In [2]:
# import os
# os.environ["HF_HOME"] = (
#     "/media/s44504/3b01c699-3670-469b-801f-13880b9cac56/huggingface/"
# )

In [3]:
# from datasets import load_dataset
# import soundfile as sf
# from pathlib import Path
# import os
# import numpy as np

# # dataset_list = ["en-US-Madison", "en-AU-Zak"] # range(0, 5)
# dataset_list = ["en-UK-Thalia"] # (range(47, 57))
# # dataset_list = ["sw-TZ-Victoria"]
# # dataset_list = ["id-ID-Althaf"]

# for dataset_uri in dataset_list:
#     dataset = load_dataset("bookbot/en-bookbot-studio", num_proc=os.cpu_count)
#     # dataset = load_dataset("bookbot/sw-TZ-Victoria", num_proc=os.cpu_count)
#     dataset = dataset.filter(lambda example: example["speaker"] == dataset_uri)
#     dataset = dataset["train"].select(range(0, 5))
#     dataset_name = Path(dataset_uri).stem

#     # Specify the directory where you want to save the WAV files
#     output_directory = f"{dataset_name}"

#     # Ensure the output directory exists
#     os.makedirs(output_directory, exist_ok=True)

#     # Initialize an empty list to hold the audio data arrays
#     concatenated_audio = []

#     # Iterate over each item in the dataset
#     for i, item in enumerate(dataset):
#         # Extract the audio array and sampling rate
#         audio_array = item["audio"]["array"]
#         sampling_rate = item["audio"]["sampling_rate"]
#         try:
#             id = item["id"]
#         except:
#             id = i

#         # Define the output file path
#         output_file_path = os.path.join(output_directory, f"{id}.wav")

#         # Write the audio data to a WAV file
#         sf.write(output_file_path, audio_array, sampling_rate)

#         # Append the audio data to the list
#         concatenated_audio.append(audio_array)

#         silence_length_samples = int((350 / 1000) * sampling_rate)
#         silence_audio = np.zeros(silence_length_samples)
#         concatenated_audio.append(silence_audio)

#         print(f"Written file {output_file_path}")

#     # Concatenate the audio data arrays
#     concatenated_audio = np.concatenate(concatenated_audio)

#     # Define the output file path for the concatenated audio
#     concatenated_file_path = os.path.join(output_directory, "concatenated_audio.wav")

#     # Write the concatenated audio data to a WAV file
#     sf.write(concatenated_file_path, concatenated_audio, sampling_rate)

#     print(f"Concatenated audio written to {concatenated_file_path}")

In [4]:
# # Iterate over each item in the dataset
# import os
# import soundfile as sf
# import numpy as np

# path_dir = "/home/s44504/StyleTTS2/Demo/id-ID-Althaf"

# # Initialize the concatenated_audio list
# concatenated_audio = []

# for i, filename in enumerate(os.listdir(path_dir)):
#     if filename.endswith(".wav"):
#         file_path = os.path.join(path_dir, filename)
#         # Load the audio file using soundfile
#         audio_array, sampling_rate = sf.read(file_path)

#         # Add 350ms of silence
#         silence_length_samples = int((400 / 1000) * sampling_rate)
#         silence_audio = np.zeros(silence_length_samples)

#         # Concatenate the audio with silence
#         audio_with_silence = np.concatenate([audio_array, silence_audio])

#         # Append the audio data with silence to the list
#         concatenated_audio.append(audio_with_silence)

# output_directory = "/home/s44504/StyleTTS2/Demo/id-ID-Althaf"

# # Concatenate all audio arrays in the list
# final_concatenated_audio = np.concatenate(concatenated_audio)

# # Define the output file path for the concatenated audio
# concatenated_file_path = os.path.join(output_directory, "concatenated_audio.wav")

# # Write the concatenated audio data to a WAV file
# sf.write(concatenated_file_path, final_concatenated_audio, sampling_rate)

# print(f"Concatenated audio written to {concatenated_file_path}")

In [5]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.cuda.set_device(1)


import random
random.seed(0)

import numpy as np
np.random.seed(0)

%cd ..

# load packages
import time
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize
import soundfile as sf
from pathlib import Path

from models import *
from utils import *
from text_utils import TextCleaner
textcleaner = TextCleaner()

%matplotlib inline

SAMPLING_RATE = 24000

/home/s44504/StyleTTS2-clone
177


In [6]:
to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300
)
mean, std = -4, 4


def length_to_mask(lengths):
    mask = (
        torch.arange(lengths.max())
        .unsqueeze(0)
        .expand(lengths.shape[0], -1)
        .type_as(lengths)
    )
    mask = torch.gt(mask + 1, lengths.unsqueeze(1))
    return mask


def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor


def compute_style(path):
    wave, sr = librosa.load(path, sr=SAMPLING_RATE)
    audio, index = librosa.effects.trim(wave, top_db=30)
    if sr != SAMPLING_RATE:
        audio = librosa.resample(audio, sr, SAMPLING_RATE)
    mel_tensor = preprocess(audio).to(device)

    with torch.no_grad():
        ref_s = model.style_encoder(mel_tensor.unsqueeze(1))
        ref_p = model.predictor_encoder(mel_tensor.unsqueeze(1))

    return torch.cat([ref_s, ref_p], dim=1)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# load phonemizer

config = yaml.safe_load(open(
    # "Models/EN-Multi-ID-Althaf-emphasis/config_ft_en_multi_id_althaf_sw_victoria.yml"
    "/home/s44504/StyleTTS2-clone/Models/EN-Multi-ID-Althaf-SW-Victoria/config_ft_en_multi_id_althaf_sw_victoria.yml"
    ))

# load pretrained ASR model
ASR_config = config.get("ASR_config", False)
ASR_path = config.get("ASR_path", False)
print(ASR_config)
print(ASR_path)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get("F0_path", False)
pitch_extractor = load_F0_models(F0_path)

# load BERT model
from Utils.PLBERT.util import load_plbert

BERT_path = config.get("PLBERT_dir", False)
plbert = load_plbert(BERT_path)

model_params = recursive_munch(config["model_params"])
model = build_model(model_params, text_aligner, pitch_extractor, plbert)
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

Utils/ASR/config.yml
Utils/ASR/epoch_00080.pth


/home/s44504/miniconda3/envs/vad/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


/home/s44504/miniconda3/envs/vad/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [8]:
params_whole = torch.load("/home/s44504/StyleTTS2-clone/Models/EN-Multi-ID-Althaf-SW-Victoria/epoch_2nd_00019.pth"
                        #   "Models/EN-Multi-ID-Althaf-emphasis/epoch_2nd_00029.pth"
                        , map_location="cpu")
params = params_whole["net"]

In [9]:
for key in model:
    if key in params:
        print("%s loaded" % key)
        try:
            model[key].load_state_dict(params[key])
        except:
            from collections import OrderedDict

            state_dict = params[key]
            
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k[7:]  # remove `module.`
                new_state_dict[name] = v
            # load params
            model[key].load_state_dict(new_state_dict, strict=False)
#             except:
#                 _load(params[key], model[key])
_ = [model[key].eval() for key in model]

bert loaded
bert_encoder loaded
predictor loaded
decoder loaded
text_encoder loaded
predictor_encoder loaded
style_encoder loaded
diffusion loaded
text_aligner loaded
pitch_extractor loaded
mpd loaded
msd loaded
wd loaded


In [10]:
from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule

sampler = DiffusionSampler(
    model.diffusion.diffusion,
    sampler=ADPM2Sampler(),
    sigma_schedule=KarrasSchedule(
        sigma_min=0.0001, sigma_max=3.0, rho=9.0
    ),  # empirical parameters
    clamp=False,
)

## Synthesize Speech


In [11]:
def inference(
    text,
    ref_s,
    global_phonemizer,
    alpha=0.3,
    beta=0.7,
    diffusion_steps=5,
    embedding_scale=1,
    phonemes=False,
):
    text = text.strip()
    if phonemes:
        ps = text
    else:
        ps = global_phonemizer.phonemize([text])[0]
    print(f"ps: {ps}")
    # ps = word_tokenize(ps[0])
    # ps = " ".join(ps)
    tokens = textcleaner(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        text_mask = length_to_mask(input_lengths).to(device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(
            noise=torch.randn((1, 256)).unsqueeze(1).to(device),
            embedding=bert_dur,
            embedding_scale=embedding_scale,
            features=ref_s,
            num_steps=diffusion_steps,
        ).squeeze(1)

        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        ref = alpha * ref + (1 - alpha) * ref_s[:, :128]
        s = beta * s + (1 - beta) * ref_s[:, 128:]

        d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)

        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame : c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device)
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(en)
            asr_new[:, :, 0] = en[:, :, 0]
            asr_new[:, :, 1:] = en[:, :, 0:-1]
            en = asr_new

        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

        asr = t_en @ pred_aln_trg.unsqueeze(0).to(device)
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(asr)
            asr_new[:, :, 0] = asr[:, :, 0]
            asr_new[:, :, 1:] = asr[:, :, 0:-1]
            asr = asr_new

        out = model.decoder(asr, F0_pred, N_pred, ref.squeeze().unsqueeze(0))

    return (
        out.squeeze().cpu().numpy()[..., :-50]
    )  # weird pulse at the end of the model, need to be fixed later

In [12]:
from pathlib import Path
import os
import soundfile as sf
import phonemizer


def synthesize_speech(reference_dicts, output_directory, file_name, language, phonemes=False, alpha=0.3, beta=0.7, diffusion_steps = 10, embedding_scale=1):
    global_phonemizer = phonemizer.backend.EspeakBackend(
        language=language, preserve_punctuation=True, with_stress=True
    )
    start = time.time()
    noise = torch.randn(1, 1, 256).to(device)
    idx = 0
    for text, path in reference_dicts.items():
        try:
            # Convert string path to Path object for easier manipulation
            ref_s = compute_style(path)
            path = Path(path)
            # Create the output directory based on the reference path
            os.makedirs(output_directory, exist_ok=True)

            wav = inference(
                text,
                ref_s,
                global_phonemizer,
                alpha=alpha,
                beta=beta,
                diffusion_steps=diffusion_steps,
                embedding_scale=embedding_scale,
                phonemes=phonemes,
            )

            rtf = (time.time() - start) / (len(wav) / SAMPLING_RATE)
            print(f"RTF = {rtf:5f}")
            import IPython.display as ipd

            # print(k + " Synthesized:")
            # display(ipd.Audio(wav, rate=SAMPLING_RATE, normalize=False))

            sf.write(f"{output_directory}/{file_name}_{idx}.wav", wav, SAMPLING_RATE)
            print(f"Audio saved to: {output_directory}/{file_name}_{idx}.wav")
            idx += 1
        except Exception as e:
            print(e)

        # print("Reference:")
        # display(ipd.Audio(path, rate=SAMPLING_RATE, normalize=False))

### US MADISON


In [30]:
reference_dicts = {
    # """ StyleTTS 2 is a text to speech model that leverages style diffusion and adversarial training with large speech language models to achieve human level text to speech synthesis. """: "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    # """ Then... whoosh! It takes off! """: "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    """ ...and a sunny yellow roof over there. """: "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    """ So he is convinced he can fix that. So he goes to work creating a red hut here... """: "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    """ He sniffs and sniffs, then ... A ... A ... A ... choo! """: "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    """ This is Forester. He works out in the forest. """: "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    """ This is Forester. He works out in the forest. It brings him a lot of happiness to work with animals. He checks on them every day to make sure that they are doing well. """: "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    """ I am a spook and I am in the mood to scare... YOU! """: "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    # # """ I am very angry at him! He pisses me off. """: "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    # "I am very angry at him! He pisses me off.": "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    # "The field of astronomy is a joke! Its theories are based on flawed observations!" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav",
    # "This is absolutely unacceptable, and I won’t tolerate it any longer!" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_audio.wav"

}

In [35]:
synthesize_speech(reference_dicts, "./Demo/en-US-Madison/madison", "madison_triple_dots", language="en-us", alpha=0.3, beta=0.7, embedding_scale=2)

ps: ...ænd ɐ sˈʌni jˈɛloʊ ɹˈuːf ˌoʊvɚ ðˈɛɹ. 
RTF = 0.041359
Audio saved to: ./Demo/en-US-Madison/madison/madison_triple_dots_0.wav
ps: sˌoʊ hiː ɪz kənvˈɪnst hiː kæn fˈɪks ðˈæt. sˌoʊ hiː ɡoʊz tə wˈɜːk kɹiːˈeɪɾɪŋ ɐ ɹˈɛd hˈʌt hˈɪɹ... 
RTF = 0.043748
Audio saved to: ./Demo/en-US-Madison/madison/madison_triple_dots_1.wav
ps: hiː snˈɪfs ænd snˈɪfs, ðˈɛn ... ˈeɪ ... ˈeɪ ... ˈeɪ ... tʃˈuː! 
RTF = 0.087299
Audio saved to: ./Demo/en-US-Madison/madison/madison_triple_dots_2.wav
ps: ðɪs ɪz fˈɔːɹɪstɚ. hiː wˈɜːks ˈaʊt ɪnðə fˈɔːɹɪst. 
RTF = 0.142168
Audio saved to: ./Demo/en-US-Madison/madison/madison_triple_dots_3.wav
ps: ðɪs ɪz fˈɔːɹɪstɚ. hiː wˈɜːks ˈaʊt ɪnðə fˈɔːɹɪst. ɪt bɹˈɪŋz hˌɪm ɐ lˈɑːt ʌv hˈæpɪnəs tə wˈɜːk wɪð ˈænɪməlz. hiː tʃˈɛks ˌɔn ðˌɛm ˈɛvɹi dˈeɪ tə mˌeɪk ʃˈʊɹ ðæt ðeɪ ɑːɹ dˌuːɪŋ wˈɛl. 
RTF = 0.071910
Audio saved to: ./Demo/en-US-Madison/madison/madison_triple_dots_4.wav
ps: aɪɐm ɐ spˈuːk ænd aɪɐm ɪnðə mˈuːd tə skˈɛɹ... juː! 
RTF = 0.247503
Audio saved to: ./Demo/en-US-Madison/madison/madi

### UK THALIA


In [84]:
reference_dicts = {
    """ StyleTTS 2 is a text to speech model that leverages style diffusion and adversarial training with large speech language models to achieve human level text to speech synthesis. """: "/home/s44504/StyleTTS2/Demo/en-UK-Thalia/concatenated_audio.wav",
    """ Then, whoosh! It takes off! """: "/home/s44504/StyleTTS2/Demo/en-UK-Thalia/concatenated_audio.wav",
    """ So he is convinced he can fix that. So he goes to work creating a red hut here. """: "/home/s44504/StyleTTS2/Demo/en-UK-Thalia/concatenated_audio.wav",
    """ This is Forester. He works out in the forest. """: "/home/s44504/StyleTTS2/Demo/en-UK-Thalia/concatenated_audio.wav",
    """ It brings him a lot of happiness to work with animals. """: "/home/s44504/StyleTTS2/Demo/en-UK-Thalia/concatenated_audio.wav",
    """ He checks on them every day to make sure that they are doing well. """: "/home/s44504/StyleTTS2/Demo/en-UK-Thalia/concatenated_audio.wav",
}

In [86]:
synthesize_speech(reference_dicts, "./Demo/en-UK-Thalia/thalia", "en-thalia", language="en-gb", alpha=0.3, beta=0.5, embedding_scale=1)

ps: stˈaɪl tˌiːtˌiːˈɛs tˈuː ɪz ɐ tˈɛkst tə spˈiːtʃ mˈɒdəl ðat lˈiːvəɹɪdʒɪz stˈaɪl dɪfjˈuːʒən and ˌadvəsˈeəɹɪəl tɹˈeɪnɪŋ wɪð lˈɑːdʒ spˈiːtʃ lˈaŋɡwɪdʒ mˈɒdəlz tʊ ɐtʃˈiːv hjˈuːmən lˈɛvəl tˈɛkst tə spˈiːtʃ sˈɪnθəsˌɪs. 
RTF = 0.017083
Audio saved to: ./Demo/en-UK-Thalia/thalia/en-thalia_0.wav
ps: ðˈɛn. wˈuːʃ! ɪt tˈeɪks ˈɒf! 
RTF = 0.141247
Audio saved to: ./Demo/en-UK-Thalia/thalia/en-thalia_1.wav
ps: sˌəʊ hiː ɪz kənvˈɪnst hiː kan fˈɪks ðˈat. sˌəʊ hiː ɡəʊz tə wˈɜːk kɹiːˈeɪtɪŋ ɐ ɹˈɛd hˈʌt hˈiə. 
RTF = 0.083683
Audio saved to: ./Demo/en-UK-Thalia/thalia/en-thalia_2.wav
ps: ðɪs ɪz fˈɒɹɪstə. hiː wˈɜːks ˈaʊt ɪnðə fˈɒɹɪst. 
RTF = 0.151766
Audio saved to: ./Demo/en-UK-Thalia/thalia/en-thalia_3.wav
ps: ɪt bɹˈɪŋz hˌɪm ɐ lˈɒt ɒv hˈapɪnəs tə wˈɜːk wɪð ˈanɪməlz. 
RTF = 0.172957
Audio saved to: ./Demo/en-UK-Thalia/thalia/en-thalia_4.wav
ps: hiː tʃˈɛks ˌɒn ðˌɛm ˈɛvɹɪ dˈeɪ tə mˌeɪk ʃˈʊə ðat ðeɪ ɑː dˌuːɪŋ wˈɛl. 
RTF = 0.183488
Audio saved to: ./Demo/en-UK-Thalia/thalia/en-thalia_5.wav


### AU Zak


In [14]:
reference_dicts = {
    """ StyleTTS 2 is a text to speech model that leverages style diffusion and adversarial training with large speech language models to achieve human level text to speech synthesis. """: "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """ Then, whoosh! It takes off! """: "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """ So he is convinced he can fix that. So he goes to work creating a red hut here. """: "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """ This is Forester. He works out in the forest. """: "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """ It brings him a lot of happiness to work with animals. """: "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """ He checks on them every day to make sure that they are doing well. """: "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """ Diya never wants to talk in class. """: "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
}

synthesize_speech(reference_dicts, "./Demo/en-AU-Zak/zak", "en-zak", language="en-gb", alpha=0.3, beta=0.5, embedding_scale=1)

ps: stˈaɪl tˌiːtˌiːˈɛs tˈuː ɪz ɐ tˈɛkst tə spˈiːtʃ mˈɒdəl ðat lˈiːvəɹɪdʒɪz stˈaɪl dɪfjˈuːʒən and ˌadvəsˈeəɹɪəl tɹˈeɪnɪŋ wɪð lˈɑːdʒ spˈiːtʃ lˈaŋɡwɪdʒ mˈɒdəlz tʊ ɐtʃˈiːv hjˈuːmən lˈɛvəl tˈɛkst tə spˈiːtʃ sˈɪnθəsˌɪs. 
RTF = 0.014692
Audio saved to: ./Demo/en-AU-Zak/zak/en-zak_0.wav
ps: ðˈɛn, wˈuːʃ! ɪt tˈeɪks ˈɒf! 
RTF = 0.179565
Audio saved to: ./Demo/en-AU-Zak/zak/en-zak_1.wav
ps: sˌəʊ hiː ɪz kənvˈɪnst hiː kan fˈɪks ðˈat. sˌəʊ hiː ɡəʊz tə wˈɜːk kɹiːˈeɪtɪŋ ɐ ɹˈɛd hˈʌt hˈiə. 
RTF = 0.093003
Audio saved to: ./Demo/en-AU-Zak/zak/en-zak_2.wav
ps: ðɪs ɪz fˈɒɹɪstə. hiː wˈɜːks ˈaʊt ɪnðə fˈɒɹɪst. 
RTF = 0.203376
Audio saved to: ./Demo/en-AU-Zak/zak/en-zak_3.wav
ps: ɪt bɹˈɪŋz hˌɪm ɐ lˈɒt ɒv hˈapɪnəs tə wˈɜːk wɪð ˈanɪməlz. 
RTF = 0.232056
Audio saved to: ./Demo/en-AU-Zak/zak/en-zak_4.wav
ps: hiː tʃˈɛks ˌɒn ðˌɛm ˈɛvɹɪ dˈeɪ tə mˌeɪk ʃˈʊə ðat ðeɪ ɑː dˌuːɪŋ wˈɛl. 
RTF = 0.243233
Audio saved to: ./Demo/en-AU-Zak/zak/en-zak_5.wav
ps: dˈɪjə nˈɛvə wˈɒnts tə tˈɔːk ɪn klˈas. 
RTF = 0.458014
Audio saved to: .

In [27]:
reference_dicts = {
    "dˈɪjə \"nˈɛv\"ə wˈɒnts tə tˈɔːk ɪn klˈas.": "/home/s44504/StyleTTS2-clone/Demo/en-AU-Zak/zak2althaf/zak2althaf_6.wav",
}

synthesize_speech(reference_dicts, "./Demo/en-AU-Zak/zak2althaf-emphasis", "en-zak2althaf-emphasis", language="en-gb", alpha=0.3, beta=0.7, embedding_scale=1, phonemes=True)

ps: dˈɪjə "nˈɛv"ə wˈɒnts tə tˈɔːk ɪn klˈas.
RTF = 0.034246
Audio saved to: ./Demo/en-AU-Zak/zak2althaf-emphasis/en-zak2althaf-emphasis_0.wav


In [ ]:
reference_dicts = {
    """That makes sense, I agree.""": "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """I completely agree with you.""": "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """That’s an interesting point.""": "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """I appreciate your insight.""": "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """I see your point, it adds value.""": "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
}

synthesize_speech(reference_dicts, "synthesized_mutilingual_en_id", "au_zak", "en-gb", phonemes=False)

ps: ðat mˌeɪks sˈɛns, aɪ ɐɡɹˈiː. 
RTF = 0.050123
ps: aɪ kəmplˈiːtli ɐɡɹˈiː wɪð juː. 
RTF = 0.093521
ps: ðats ɐn ˈɪntɹɛstɪŋ pˈɔɪnt. 
RTF = 0.175764
ps: aɪ ɐpɹˈiːʃɪˌeɪt jɔːɹ ˈɪnsaɪt. 
RTF = 0.193162
ps: aɪ sˈiː jɔː pˈɔɪnt, ɪt ˈadz vˈaljuː. 
RTF = 0.205829


### Emphasis


In [ ]:
reference_dicts = {
    """ðæt "mˌeɪks" "sˈɛns", aɪ ɐ"ɡɹˈiː".""": "/home/s44504/StyleTTS2/Demo/en-US-Madison/concatenated_audio.wav",
    """aɪ kəm"plˈiːt"li ɐ"ɡɹˈiː" wɪð juː.""": "/home/s44504/StyleTTS2/Demo/en-US-Madison/concatenated_audio.wav",
    """ðæts ɐn "ˈɪn"tɹɛstɪŋ "pˈɔɪnt".""": "/home/s44504/StyleTTS2/Demo/en-US-Madison/concatenated_audio.wav",
    """aɪ ɐ"pɹˈiː"ʃɪˌeɪt jʊɹ "ˈɪn"saɪt.""": "/home/s44504/StyleTTS2/Demo/en-US-Madison/concatenated_audio.wav",
    """aɪ "sˈiː" jʊɹ "pˈɔɪnt", ɪɾ "ˈædz" "vˈæl"juː.""": "/home/s44504/StyleTTS2/Demo/en-US-Madison/concatenated_audio.wav",
}

In [ ]:
# reference_dicts = {
#     """ This is without emphasis. This is with emphasis. """: "/home/s44504/StyleTTS2/Demo/en-AU-Zak/en-AU-Zak_0.wav",
# }

# synthesize_speech(reference_dicts, "synthesized_zak_with_emphasis", "emphasis", language="en-gb", phonemes=False)

reference_dicts = {
    """ ðɪs ɪz wɪðˌaʊt ˈɛmfəsɪs. "ðɪs" ɪz "wɪð" "ˈɛm"fəsɪs. """: "/home/s44504/StyleTTS2/Demo/en-AU-Zak/en-AU-Zak_0.wav",
}

synthesize_speech(reference_dicts, "synthesized_zak_with_emphasis", "emphasis", language="en-gb", phonemes=True)

ps: ðɪs ɪz wɪðˌaʊt ˈɛmfəsɪs. "ðɪs" ɪz "wɪð" "ˈɛm"fəsɪs.
RTF = 0.029405


In [ ]:
reference_dicts = {
    """ ðɪs ɪz wɪðˌaʊt ˈɛmfəsɪs. "ðɪs" ɪz "wɪð" "ˈɛm"fəsɪs. """: "/home/s44504/StyleTTS2/Demo/en-AU-Zak/concatenated_audio.wav",
    """aɪ kəm"plˈiːt"li ɐ"ɡɹˈiː" wɪð juː.""": "/home/s44504/StyleTTS2/Demo/en-AU-Zak/Zak2Althaf/Althaf-Zak_1.wav",
}

synthesize_speech(reference_dicts, "emphasis-althaf-zak", "emphasis-althaf-zak", language="en-gb", phonemes=True)

ps: ðɪs ɪz wɪðˌaʊt ˈɛmfəsɪs. "ðɪs" ɪz "wɪð" "ˈɛm"fəsɪs.
RTF = 0.033094
ps: aɪ kəm"plˈiːt"li ɐ"ɡɹˈiː" wɪð juː.
RTF = 0.109920


#### Emotion

In [ ]:
reference_dicts = {
    """ This is without emphasis. This is with emphasis. """: "/home/s44504/StyleTTS2-Demo/reference_audio/zak_emotions/Angry-1.wav",
}

synthesize_speech(reference_dicts, "synthesized_zak_angry", "angry", language="en-gb", phonemes=False)

### ID ALTHAF


In [ ]:
reference_dicts = {}

dir_path = Path("Demo/id-ID-Althaf")

# Iterate through each file in the directory
for k, file_path in enumerate(sorted(dir_path.iterdir())):
    if file_path.is_file():
        reference_dicts[k] = file_path

In [ ]:
synthesize_speech(reference_dicts)

'int' object has no attribute 'strip'
'int' object has no attribute 'strip'
'int' object has no attribute 'strip'
'int' object has no attribute 'strip'
'int' object has no attribute 'strip'
'int' object has no attribute 'strip'


### SW Victoria


In [1]:
from datasets import load_dataset

dataset = load_dataset("bookbot/OpenBible_Swahili", "JHN_clean", num_proc=20)

Setting num_proc from 20 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
from IPython.display import display, Audio
import soundfile as sf

def play_audio(example):
    audio = example['audio']
    display(Audio(audio['array'], rate=audio['sampling_rate']))
    
play_audio(dataset["train"][0])
output_directory = "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original"
os.makedirs(output_directory, exist_ok=True)


for i, datum in enumerate(dataset["train"]):
    if i == 6:
        break
    audio_sample = datum["audio"]

        # Create the full path for the output file
    output_file_path = os.path.join(output_directory, f"{datum['id']}.wav")

    # Write the audio data to the file
    sf.write(output_file_path, audio_sample["array"], audio_sample["sampling_rate"])
    

In [21]:
dataset["train"][4]

{'id': 'JHN_001_005',
 'verse_id': 'JHN 1:5',
 'audio': {'path': '/media/s44504/3b01c699-3670-469b-801f-13880b9cac56/huggingface/datasets/downloads/extracted/a3debecf178e71517a97dcd6a78c54bb793815bd5a187a59d741f32acf9eff1f/JHN/JHN_001/JHN_001_005.wav',
  'array': array([ 0.26275635,  0.26443481,  0.2628479 , ...,  0.01086426,
         -0.00314331, -0.02203369]),
  'sampling_rate': 44100},
 'verse_text': 'Nuru hungʼaa gizani nalo giza halikuishinda.',
 'transcript': 'nuru hungaa gizani nalo giza halikuishinda'}

In [66]:
global_phonemizer = phonemizer.backend.EspeakBackend(
        language="sw", preserve_punctuation=True, with_stress=True
)

global_phonemizer.phonemize(["tanggu"])

['tˈaŋɡɡu ']

In [68]:
from gruut import sentences 
for words in sentences("tangu", lang="sw"):
    for word in words:
        print(word.phonemes)

['t', 'ɑ', 'ᵑg', 'u']


In [47]:
from word_info_sw import get_word_info

get_word_info(["hung'aa"])

[{'word': "hung'aa",
  'syllable': "hu.ng'aa",
  'ipa': 'hu.ŋɑɑ',
  'description': ''}]

In [47]:
reference_dicts = {
    # """ hapo mwanzo alikuwako neno huyo neno alikuwa pamoja na mungu naye neno alikuwa mungu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original/JHN_001_001.wav",
    # """ tangu mwanzo huyo neno alikuwa pamoja na mungu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original/JHN_001_002.wav",
    # """ vitu vyote viliumbwa kwa yeye wala pasipo yeye hakuna chochote kilichoumbwa ambacho kimeumbwa. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original/JHN_001_003.wav",
    """ ndani, yake ndimo. ulimokuwa uzima na huo uzima ulikuwa nuru ya watu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original/JHN_001_004.wav",
    # """ nuru hung'aa gizani nalo giza halikuishinda. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original/JHN_concatenated.wav",
}

# reference_dicts = {
#     """hˈapo mwˈanzo ˌalikuwˈako nˈeno hˈujo nˈeno ˌalikˈuwa pamˈoɟa na mˈuŋɡu nˌaje nˈeno ˌalikˈuwa mˈuŋɡu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/JHN_001_001.wav",
#     """tɑŋɡu mwˈanzo hˈujo nˈeno ˌalikˈuwa pamˈoɟa na muᵑgu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/JHN_001_002.wav",
#     """vitu vyote viliumbwa kwa yeye wala pasipo yeye hakuna chochote kilichoumbwa ambacho kimeumbwa. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/JHN_001_003.wav",
#     """ndani yake ndimo ulimokuwa uzima na huo uzima ulikuwa nuru ya watu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/JHN_001_004.wav",
#     """nuru huŋɑɑ gizani nalo giza halikuishinda. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/JHN_001_005.wav",
# }

synthesize_speech(reference_dicts, "./Demo/Swahili-Bible-John/synthesized-with-pauses", "sw-john-no-pauses", language="sw", alpha=0.3, beta=0.5, embedding_scale=1)

ps: ndˈani,, jˌake ndˈimo. ˌulimokˈuwa uzˈima na hˈuo uzˈima ˌulikˈuwa nˈuru ja wˈatu. 
RTF = 0.014577
Audio saved to: ./Demo/Swahili-Bible-John/synthesized-with-pauses/sw-john-no-pauses_0.wav


In [55]:
reference_dicts = {
    """hˈapo mwˈanzo ˌalikuwˈako nˈeno hˈujo nˈeno ˌalikˈuwa pamˈoɟa na mˈuŋɡu nˌaje nˈeno ˌalikˈuwa mˈuŋɡu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original/JHN_001_001.wav",
    """tɑŋɡu mwˈanzo hˈujo nˈeno ˌalikˈuwa pamˈoɟa na mˈuŋɡu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original/JHN_001_002.wav",
    """vˈitu vjˈote vˌiliˈumbwa kwa jˌeje wˈala pasˈipo jˌeje hakˈuna tʃotʃˈote kˌilitʃoˈumbwa ambˈatʃo kˌimeˈumbwa.  """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original/JHN_001_003.wav",
    """ndˈani jˌake ndˈimo ˌulimokˈuwa uzˈima na hˈuo uzˈima ˌulikˈuwa nˈuru ja wˈatu.. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original/JHN_001_004.wav",
    """nˈuru huŋˈaa ɡizˈani nˈalo ɡˈiza hˌalikˌuiʃˈinda. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/original/JHN_001_005.wav",
}

synthesize_speech(reference_dicts, "./Demo/Swahili-Bible-John/synthesized-phonemes-v2", "sw-john", language="sw", alpha=0.3, beta=0.7, embedding_scale=1, phonemes=True)

ps: hˈapo mwˈanzo ˌalikuwˈako nˈeno hˈujo nˈeno ˌalikˈuwa pamˈoɟa na mˈuŋɡu nˌaje nˈeno ˌalikˈuwa mˈuŋɡu.
RTF = 0.015432
Audio saved to: ./Demo/Swahili-Bible-John/synthesized-phonemes-v2/sw-john_0.wav
ps: tɑŋɡu mwˈanzo hˈujo nˈeno ˌalikˈuwa pamˈoɟa na mˈuŋɡu.
RTF = 0.046252
Audio saved to: ./Demo/Swahili-Bible-John/synthesized-phonemes-v2/sw-john_1.wav
ps: vˈitu vjˈote vˌiliˈumbwa kwa jˌeje wˈala pasˈipo jˌeje hakˈuna tʃotʃˈote kˌilitʃoˈumbwa ambˈatʃo kˌimeˈumbwa.
RTF = 0.044663
Audio saved to: ./Demo/Swahili-Bible-John/synthesized-phonemes-v2/sw-john_2.wav
ps: ndˈani jˌake ndˈimo ˌulimokˈuwa uzˈima na hˈuo uzˈima ˌulikˈuwa nˈuru ja wˈatu..
RTF = 0.077457
Audio saved to: ./Demo/Swahili-Bible-John/synthesized-phonemes-v2/sw-john_3.wav
ps: nˈuru huŋˈaa ɡizˈani nˈalo ɡˈiza hˌalikˌuiʃˈinda.
RTF = 0.136244
Audio saved to: ./Demo/Swahili-Bible-John/synthesized-phonemes-v2/sw-john_4.wav


In [ ]:
# for f in *.wav; do ffmpeg -i "$f" -vn -ar 24000 -ac 2 -b:a 192k "${f%.wav}_converted.mp3"; done

## S2S


In [ ]:
# def s2s_emotion(
#     text,
#     source_s,
#     target_s,
#     language,
#     alpha=0.8,
#     beta=0.1,
#     diffusion_steps=10,
#     embedding_scale=1,
#     phonemes=False,
# ):
#     global_phonemizer = phonemizer.backend.EspeakBackend(
#         language=language, preserve_punctuation=True, with_stress=True
#     )

#     text = text.strip()
#     if phonemes:
#         ps = text
#     else:
#         ps = global_phonemizer.phonemize([text])[0]
#     print(f"ps: {ps}")
#     tokens = textcleaner(ps)
#     tokens.insert(0, 0)
#     tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

#     with torch.no_grad():
#         input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
#         text_mask = length_to_mask(input_lengths).to(device)

#         t_en = model.text_encoder(tokens, input_lengths, text_mask)
#         bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
#         d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

#         # style encoder
#         s_pred = sampler(
#             noise=torch.randn((1, 256)).unsqueeze(1).to(device),
#             embedding=bert_dur,
#             embedding_scale=embedding_scale,
#             features=source_s, 
#             num_steps=diffusion_steps,
#         ).squeeze(1)
        
#         s_pred_target = sampler(
#             noise=torch.randn((1, 256)).unsqueeze(1).to(device),
#             embedding=bert_dur,
#             embedding_scale=embedding_scale,
#             features=target_s,  
#             num_steps=diffusion_steps,
#         ).squeeze(1)

#         ref = s_pred[:, :128]
#         s = s_pred[:, 128:]

#         ref = alpha * ref + (1 - alpha) * target_s[:, :128]
#         s = beta * s + (1 - beta) * target_s[:, 128:]

#         # text encoder
#         d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)

#         x, _ = model.predictor.lstm(d)
#         duration = model.predictor.duration_proj(x)

#         duration = torch.sigmoid(duration).sum(axis=-1)
#         pred_dur = torch.round(duration.squeeze()).clamp(min=1)

#         pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
#         c_frame = 0
#         for i in range(pred_aln_trg.size(0)):
#             pred_aln_trg[i, c_frame : c_frame + int(pred_dur[i].data)] = 1
#             c_frame += int(pred_dur[i].data)

#         # encode prosody
#         en = d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device)
#         if model_params.decoder.type == "hifigan":
#             asr_new = torch.zeros_like(en)
#             asr_new[:, :, 0] = en[:, :, 0]
#             asr_new[:, :, 1:] = en[:, :, 0:-1]
#             en = asr_new

#         F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

#         asr = t_en @ pred_aln_trg.unsqueeze(0).to(device)
#         if model_params.decoder.type == "hifigan":
#             asr_new = torch.zeros_like(asr)
#             asr_new[:, :, 0] = asr[:, :, 0]
#             asr_new[:, :, 1:] = asr[:, :, 0:-1]
#             asr = asr_new

#         out = model.decoder(asr, F0_pred, N_pred, ref.squeeze().unsqueeze(0))

#     return out.squeeze().cpu().numpy()[..., :-50]


# ### Angry
# reference_dicts = {
#     "I am very angry at him! He pisses me off" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison2althaf/madison2althaf_angry_0.wav",
#     "The field of astronomy is a joke! Its theories are based on flawed observations!" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison2althaf/madison2althaf_angry_1.wav",
#     "This is absolutely unacceptable, and I won’t tolerate it any longer!" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison2althaf/madison2althaf_angry_2.wav"
# }

# target_emotion = compute_style(
#     "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_angry_audio.wav"
#     # "/home/s44504/StyleTTS2/Demo/id-ID-Althaf/concatenated_audio.wav"
#     # "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison2althaf/madison2althaf_0.wav"
# )

# idx = 0
# for text, ref_source in reference_dicts.items():
#     source_style = compute_style(ref_source)
#     wav = s2s_emotion(
#         text,
#         source_style,
#         target_emotion,
#         "en-us",
#         alpha=0.1,
#         beta=0.9,
#         diffusion_steps=10,
#         embedding_scale=2,
#         phonemes=False,
#     )
    
#     target_dir = Path("Demo/en-US-Madison/madison2althaf_angry_v2")
#     os.makedirs(target_dir, exist_ok=True)
#     sf.write(f"{target_dir}/madison2althaf_{idx}.wav", wav, SAMPLING_RATE)
#     idx += 1

ps: aɪɐm vˈɛɹi ˈæŋɡɹi æt hˌɪm! hiː pˈɪsᵻz mˌiː ˈɔf 
ps: ðə fˈiːld ʌv ɐstɹˈɑːnəmi ɪz ɐ dʒˈoʊk! ɪts θˈiəɹiz ɑːɹ bˈeɪst ˌɔn flˈɔːd ɑːbzɚvˈeɪʃənz! 
ps: ðɪs ɪz ˌæbsəlˈuːtli ʌnɐksˈɛptəbəl, ænd aɪ woʊnt tˈɑːlɚɹˌeɪt ɪɾ ˌɛni lˈɔŋɡɚ! 


In [57]:
def s2s(
    text,
    ref_s,
    target_s,
    language,
    alpha=0.8,
    beta=0.1,
    diffusion_steps=10,
    embedding_scale=1,
    phonemes=False,
):
    global_phonemizer = phonemizer.backend.EspeakBackend(
        language=language, preserve_punctuation=True, with_stress=True
    )

    text = text.strip()
    if phonemes:
        ps = text
    else:
        ps = global_phonemizer.phonemize([text])[0]
    print(f"ps: {ps}")
    # ps = word_tokenize(ps[0])
    # ps = " ".join(ps)
    tokens = textcleaner(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        text_mask = length_to_mask(input_lengths).to(device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(
            noise=torch.randn((1, 256)).unsqueeze(1).to(device),
            embedding=bert_dur,
            embedding_scale=embedding_scale,
            features=target_s,  # reference from the same speaker as the embedding
            num_steps=diffusion_steps,
        ).squeeze(1)

        ref = s_pred[:, :128]
        s = s_pred[:, 128:]
        
        # Ref depebds on target_styke
        ref = alpha * ref + (1 - alpha) * ref_s[:, :128]
        s = beta * s + (1 - beta) * ref_s[:, 128:]

        d = model.predictor.text_encoder(d_en, s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)

        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame : c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device)
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(en)
            asr_new[:, :, 0] = en[:, :, 0]
            asr_new[:, :, 1:] = en[:, :, 0:-1]
            en = asr_new

        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

        asr = t_en @ pred_aln_trg.unsqueeze(0).to(device)
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(asr)
            asr_new[:, :, 0] = asr[:, :, 0]
            asr_new[:, :, 1:] = asr[:, :, 0:-1]
            asr = asr_new

        out = model.decoder(asr, F0_pred, N_pred, ref.squeeze().unsqueeze(0))

    return out.squeeze().cpu().numpy()[..., :-50]

In [61]:
### Angry
# reference_dicts = {
#     # "StyleTTS 2 is a text to speech model that leverages style diffusion and adversarial training with large speech language models to achieve human level text to speech synthesis." : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison2althaf/madison2althaf_0.wav",
#     # "I am very angry at him! He pisses me off" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison2althaf/madison2althaf_1.wav",
#     "The field of astronomy is a joke! Its theories are based on flawed observations!" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/Angry-1.wav",
#     "This is absolutely unacceptable, and I won’t tolerate it any longer!" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/Angry-3.wav",
#     "Not at this particular case, Tom, apologized Whittemore." : "/home/s44504/StyleTTS2-clone/Demo/jeanie-Angry/amused_1-28_0002.wav",
# }

# ### Thalia
# reference_dicts = {
#     """ StyleTTS 2 is a text to speech model that leverages style diffusion and adversarial training with large speech language models to achieve human level text to speech synthesis. """: "/home/s44504/StyleTTS2-clone/Demo/en-UK-Thalia/thalia/en-thalia_0.wav",
#     """ Then. whoosh! It takes off! """: "/home/s44504/StyleTTS2-clone/Demo/en-UK-Thalia/thalia/en-thalia_1.wav",
#     """ So he is convinced he can fix that. So he goes to work creating a red hut here. """: "/home/s44504/StyleTTS2-clone/Demo/en-UK-Thalia/thalia/en-thalia_2.wav",
#     """ This is Forester. He works out in the forest. """: "/home/s44504/StyleTTS2-clone/Demo/en-UK-Thalia/thalia/en-thalia_3.wav",
#     """ It brings him a lot of happiness to work with animals. """: "/home/s44504/StyleTTS2-clone/Demo/en-UK-Thalia/thalia/en-thalia_4.wav",
#     """ He checks on them every day to make sure that they are doing well. """: "/home/s44504/StyleTTS2-clone/Demo/en-UK-Thalia/thalia/en-thalia_5.wav",
# }

# ### Zak
# reference_dicts = {
#     """ StyleTTS 2 is a text to speech model that leverages style diffusion and adversarial training with large speech language models to achieve human level text to speech synthesis. """: "/home/s44504/StyleTTS2-clone/Demo/en-AU-Zak/zak/en-zak_0.wav",
#     """ Then, whoosh! It takes off! """: "/home/s44504/StyleTTS2-clone/Demo/en-AU-Zak/zak/en-zak_1.wav",
#     """ So he is convinced he can fix that. So he goes to work creating a red hut here. """: "/home/s44504/StyleTTS2-clone/Demo/en-AU-Zak/zak/en-zak_2.wav",
#     """ This is Forester. He works out in the forest. """: "/home/s44504/StyleTTS2-clone/Demo/en-AU-Zak/zak/en-zak_3.wav",
#     """ It brings him a lot of happiness to work with animals. """: "/home/s44504/StyleTTS2-clone/Demo/en-AU-Zak/zak/en-zak_4.wav",
#     """ He checks on them every day to make sure that they are doing well. """: "/home/s44504/StyleTTS2-clone/Demo/en-AU-Zak/zak/en-zak_5.wav",
#     """ Diya never wants to talk in class. """: "/home/s44504/StyleTTS2-clone/Demo/en-AU-Zak/zak/en-zak_6.wav",
#     }

### Swahili
# reference_dicts = {
#     """ hapo mwanzo alikuwako neno huyo neno alikuwa pamoja na mungu naye neno alikuwa mungu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/synthesized/sw-john_0.wav",
#     """ tangu mwanzo huyo neno alikuwa pamoja na mungu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/synthesized/sw-john_1.wav",
#     """ vitu vyote viliumbwa kwa yeye wala pasipo yeye hakuna chochote kilichoumbwa ambacho kimeumbwa. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/synthesized/sw-john_2.wav",
#     """ ndani yake ndimo ulimokuwa uzima na huo uzima ulikuwa nuru ya watu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/synthesized/sw-john_3.wav",
#     """ nuru hungaa gizani nalo giza halikuishinda. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/synthesized/sw-john_4.wav",
# }

# reference_dicts = {
#     """ hapo mwanzo alikuwako neno huyo neno alikuwa pamoja na mungu naye neno alikuwa mungu """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/JHN_001_001.wav",
#     """ tangu mwanzo huyo neno alikuwa pamoja na mungu """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/JHN_001_002.wav",
#     """ vitu vyote viliumbwa kwa yeye wala pasipo yeye hakuna chochote kilichoumbwa ambacho kimeumbwa """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/JHN_001_003.wav",
#     """ ndani yake ndimo ulimokuwa uzima na huo uzima ulikuwa nuru ya watu """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/JHN_001_004.wav",
#     """ nuru hungaa gizani nalo giza halikuishinda """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/JHN_001_005.wav",
# }

reference_dicts = {
    """hˈapo mwˈanzo ˌalikuwˈako nˈeno hˈujo nˈeno ˌalikˈuwa pamˈoɟa na mˈuŋɡu nˌaje nˈeno ˌalikˈuwa mˈuŋɡu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/synthesized-phonemes-v2/sw-john_0.wav",
    """tɑŋɡu mwˈanzo hˈujo nˈeno ˌalikˈuwa pamˈoɟa na mˈuŋɡu. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/synthesized-phonemes-v2/sw-john_1.wav",
    """vˈitu vjˈote vˌiliˈumbwa kwa jˌeje wˈala pasˈipo jˌeje hakˈuna tʃotʃˈote kˌilitʃoˈumbwa ambˈatʃo kˌimeˈumbwa.  """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/synthesized-phonemes-v2/sw-john_2.wav",
    """ndˈani jˌake ndˈimo ˌulimokˈuwa uzˈima na hˈuo uzˈima ˌulikˈuwa nˈuru ja wˈatu.. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/synthesized-phonemes-v2/sw-john_3.wav",
    """nˈuru huŋˈaa ɡizˈani nˈalo ɡˈiza hˌalikˌuiʃˈinda. """: "/home/s44504/StyleTTS2-clone/Demo/Swahili-Bible-John/synthesized-phonemes-v2/sw-john_4.wav",
}

ref_althaf = compute_style(
    # "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/concatenated_angry_audio.wav"
    "/home/s44504/StyleTTS2/Demo/id-ID-Althaf/concatenated_audio.wav"
    # "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison2althaf/madison2althaf_0.wav"
)

idx = 0
for text, ref_path in reference_dicts.items():
    ref_path = compute_style(ref_path)
    wav = s2s(
        text,
        ref_path,
        ref_althaf,
        "sw",
        alpha=0.6, # The higher it is the more it is similar to the target style
        beta=0.4, # The lower it is the more it keeps the target style emotion
        diffusion_steps=10,
        phonemes=True
    )
    
    target_dir = Path("Demo/Swahili-Bible-John/sw-bible2althaf-phonemes-0.9-0.1")
    os.makedirs(target_dir, exist_ok=True)
    sf.write(f"{target_dir}/sw-bible2althaf_{idx}.wav", wav, SAMPLING_RATE)
    idx += 1

ps: hˈapo mwˈanzo ˌalikuwˈako nˈeno hˈujo nˈeno ˌalikˈuwa pamˈoɟa na mˈuŋɡu nˌaje nˈeno ˌalikˈuwa mˈuŋɡu.
ps: tɑŋɡu mwˈanzo hˈujo nˈeno ˌalikˈuwa pamˈoɟa na mˈuŋɡu.
ps: vˈitu vjˈote vˌiliˈumbwa kwa jˌeje wˈala pasˈipo jˌeje hakˈuna tʃotʃˈote kˌilitʃoˈumbwa ambˈatʃo kˌimeˈumbwa.
ps: ndˈani jˌake ndˈimo ˌulimokˈuwa uzˈima na hˈuo uzˈima ˌulikˈuwa nˈuru ja wˈatu..
ps: nˈuru huŋˈaa ɡizˈani nˈalo ɡˈiza hˌalikˌuiʃˈinda.


In [ ]:
reference_dicts = {
    "StyleTTS 2 is a text to speech model that leverages style diffusion and adversarial training with large speech language models to achieve human level text to speech synthesis." : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison/madison_template_text_0.wav",
    # "I am very angry at him! He pisses me off" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison/madison_0.wav",
    # "The field of astronomy is a joke! Its theories are based on flawed observations!" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison/madison_1.wav",
    # "This is absolutely unacceptable, and I won’t tolerate it any longer!" : "/home/s44504/StyleTTS2-clone/Demo/en-US-Madison/madison/madison_2.wav"
}

ref_althaf = compute_style(
    "/home/s44504/StyleTTS2/Demo/id-ID-Althaf/concatenated_audio.wav"
)

idx = 0
for text, ref_path in reference_dicts.items():
    ref_path = compute_style(ref_path)
    wav = s2s(
        text,
        ref_path,
        ref_althaf,
        "en-us",
        alpha=0.6,
        beta=0.4,
        diffusion_steps=10,
        phonemes=False,
    )
    
    target_dir = Path("Demo/en-US-Madison/madison2althaf")
    os.makedirs(target_dir, exist_ok=True)
    sf.write(f"{target_dir}/madison2althaf_{idx}.wav", wav, SAMPLING_RATE)
    idx += 1

ps: stˈaɪl tˌiːtˌiːˈɛs tˈuː ɪz ɐ tˈɛkst tə spˈiːtʃ mˈɑːdəl ðæt lˈɛvɚɹɪdʒᵻz stˈaɪl dᵻfjˈuːʒən ænd ˌædvɚsˈɛɹɪəl tɹˈeɪnɪŋ wɪð lˈɑːɹdʒ spˈiːtʃ lˈæŋɡwɪdʒ mˈɑːdəlz tʊ ɐtʃˈiːv hjˈuːmən lˈɛvəl tˈɛkst tə spˈiːtʃ sˈɪnθəsˌɪs. 
